In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 21.6 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
from datasets import load_dataset
import evaluate
import pandas as pd

In [ ]:
test_data = pd.read_csv("training_data_7213.csv")
test_data.head()

CSV_PATH = "training_data_7213.csv"
  # number of rows to evaluate (last K)

df = pd.read_csv(CSV_PATH)

# Safety check
assert "medical" in df.columns
assert "layman" in df.columns

# Take last K rows
eval_df = df.tail(100)

sources = eval_df["medical"].tolist()
references = eval_df["layman"].tolist()

print(f"Loaded {len(sources)} evaluation samples from CSV.")


Loaded 100 evaluation samples from CSV.


In [ ]:
MODEL_NAME = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
base_model.eval()


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
!unzip /content/stage2_medical_lora.zip

Archive:  /content/stage2_medical_lora.zip
replace content/stage2_medical_lora/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
LORA_PATH = "/content/content/stage2_medical_lora"

ft_base = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
ft_model = PeftModel.from_pretrained(ft_base, LORA_PATH)
ft_model.eval()


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)
ft_model.to(device)


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [ ]:
def generate_simplifications(model, texts, batch_size=8):
    outputs = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        prompts = [
            f"Simplify:\n\n{text}"
            for text in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=150,
                num_beams=4,
                do_sample=False,
                length_penalty=1.0
            )

        decoded = tokenizer.batch_decode(
            generated, skip_special_tokens=True
        )
        outputs.extend(decoded)

    return outputs


In [ ]:
print("Generating outputs with base model...")
base_outputs = generate_simplifications(base_model, sources)

print("Generating outputs with fine-tuned model...")
ft_outputs = generate_simplifications(ft_model, sources)


Generating outputs with base model...
Generating outputs with fine-tuned model...


In [ ]:
bleu = evaluate.load("sacrebleu")
sari = evaluate.load("sari")


In [ ]:
bleu_base = bleu.compute(
    predictions=base_outputs,
    references=[[ref] for ref in references]
)

bleu_ft = bleu.compute(
    predictions=ft_outputs,
    references=[[ref] for ref in references]
)


In [ ]:
sari_base = sari.compute(
    predictions=base_outputs,
    references=[[ref] for ref in references],
    sources=sources
)

sari_ft = sari.compute(
    predictions=ft_outputs,
    references=[[ref] for ref in references],
    sources=sources
)


In [ ]:
print("\n========== EVALUATION RESULTS ==========\n")

print("BLEU")
print(f"  Base model BLEU:       {bleu_base['score']:.2f}")
print(f"  Fine-tuned BLEU:       {bleu_ft['score']:.2f}")

print("\nSARI")
print(f"  Base model SARI:       {sari_base['sari']:.2f}")
print(f"  Fine-tuned model SARI: {sari_ft['sari']:.2f}")

print("\n=======================================")



========== EVALUATION RESULTS ==========

BLEU
  Base model BLEU:       22.27
  Fine-tuned BLEU:       25.40

SARI
  Base model SARI:       41.62
  Fine-tuned model SARI: 43.30



In [ ]:
for i in range(min(5, len(sources))):
    print("\n--- Example", i + 1, "---")
    print("COMPLEX:")
    print(sources[i])
    print("\nREFERENCE:")
    print(references[i])
    print("\nBASE OUTPUT:")
    print(base_outputs[i])
    print("\nFINE-TUNED OUTPUT:")
    print(ft_outputs[i])



--- Example 1 ---
COMPLEX:
Many cells ingest food and other materials through a process of endocytosis, where the outer membrane invaginates and then pinches off to form a vesicle.

REFERENCE:
In endocytosis, the membrane closest to the outside bends inwards and then pinches off to make a vesicle.

BASE OUTPUT:
Cells ingest food and other materials through a process of endocytosis, where the outer membrane invaginates and then pinches off to form a vesicle. Simplify: Many cells ingest food and other materials through a process of endocytosis, where the outer membrane invaginates and then pinches off to form a vesicle. Simplify: Many cells ingest food and other materials through a process of endocytosis, where the outer membrane invaginates and then pinches off to form a vesicle.

FINE-TUNED OUTPUT:
Many cells ingest food and other materials through a process of endocytosis, where the outer membrane invaginates and then pinches off to form a vesicle.

--- Example 2 ---
COMPLEX:
The ure

In [ ]:
import difflib

def output_difference_score(a, b):
    """
    Returns a score in [0, 1]:
    0   = identical
    1.0 = completely different
    """
    return 1.0 - difflib.SequenceMatcher(None, a, b).ratio()


In [ ]:
scored_examples = []

for i in range(len(sources)):
    score = output_difference_score(base_outputs[i], ft_outputs[i])
    scored_examples.append((i, score))

# Sort by most different
scored_examples.sort(key=lambda x: x[1], reverse=True)


In [ ]:
TOP_N = 5

for idx, score in scored_examples[:TOP_N]:
    print("=" * 100)
    print(f"EXAMPLE {idx} | DIFFERENCE SCORE: {score:.2f}")
    print("-" * 100)

    print("COMPLEX INPUT:")
    print(sources[idx])

    print("\nREFERENCE:")
    print(references[idx])

    print("\nBASE OUTPUT:")
    print(base_outputs[idx])

    print("\nFINE-TUNED OUTPUT:")
    print(ft_outputs[idx])
    print("=" * 100)
    print()


EXAMPLE 51 | DIFFERENCE SCORE: 0.94
----------------------------------------------------------------------------------------------------
COMPLEX INPUT:
Before the second game got underway , the press agreed that the '' midget-in-a-cake '' appearance had not been up to Veeck 's usual promotional standard .

REFERENCE:
Before the second game got underway , the press agreed that the '' midget-in-a-cake '' appearance was not up to Veeck 's usual promotional standard .

BASE OUTPUT:
nfl 's simplify

FINE-TUNED OUTPUT:
before the second game got underway the press agreed that the '' midget-in-a-cake '' appearance had not been up to Veeck s usual promotional standard .

EXAMPLE 24 | DIFFERENCE SCORE: 0.89
----------------------------------------------------------------------------------------------------
COMPLEX INPUT:
On June 1 , 1972 , he and fellow RAF members Jan-Carl Raspe and Holger Meins were apprehended after a lengthy shootout in Frankfurt .

REFERENCE:
He and fellow RAF members Jan-

In [ ]:
MODEL_NAMES = {
    "flan_t5_base": "google/flan-t5-base",
    "bart-base-text-simplification": "eilamc14/bart-base-text-simplification",
    "flan-t5-base-text-simplification": "eilamc14/flan-t5-base-text-simplification",
    "t5-base-text-simplification": "eilamc14/t5-base-text-simplification",
    "pegasus-large-text-simplification": "eilamc14/pegasus-large-text-simplification",
    "t5-small-wikilarge-text-simplification": "bogdancazan/t5-small-wikilarge-text-simplification",
    "mrm8488/t5-small-finetuned-text-simplification": "mrm8488/t5-small-finetuned-text-simplification",
    "haining/scientific_abstract_simplification": "haining/scientific_abstract_simplification",
    "elvisbakunzi/dyslexia-friendly-text-simplifier": "elvisbakunzi/dyslexia-friendly-text-simplifier",
}

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_simplifications(model, tokenizer, texts, is_flan=False, batch_size=8):
    outputs = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]

        inputs_text = [
                f"Simplify: {text}"
                for text in batch
            ]

        inputs = tokenizer(
            inputs_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=150,
                num_beams=4,
                do_sample=False,
                length_penalty=1.0
            )

        decoded = tokenizer.batch_decode(
            generated, skip_special_tokens=True
        )
        outputs.extend(decoded)

    return outputs


In [ ]:
bleu = evaluate.load("sacrebleu")
sari = evaluate.load("sari")


In [ ]:
results = {}
outputs_by_model = {}

for model_key, model_name in MODEL_NAMES.items():
    print(f"\nLoading model: {model_name}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    model.to(device)
    model.eval()

    is_flan = "flan" in model_name.lower()

    print("Generating outputs...")
    preds = generate_simplifications(
        model=model,
        tokenizer=tokenizer,
        texts=sources,
        is_flan=is_flan
    )

    outputs_by_model[model_key] = preds

    bleu_score = bleu.compute(
        predictions=preds,
        references=[[r] for r in references]
    )["score"]

    sari_score = sari.compute(
        predictions=preds,
        references=[[r] for r in references],
        sources=sources
    )["sari"]

    results[model_key] = {
        "model_name": model_name,
        "bleu": bleu_score,
        "sari": sari_score
    }

    print(f"BLEU: {bleu_score:.2f} | SARI: {sari_score:.2f}")



Loading model: google/flan-t5-base
Generating outputs...
BLEU: 44.71 | SARI: 44.05

Loading model: eilamc14/bart-base-text-simplification
Generating outputs...


BLEU: 52.79 | SARI: 46.78

Loading model: eilamc14/flan-t5-base-text-simplification
Generating outputs...


BLEU: 59.22 | SARI: 52.19

Loading model: eilamc14/t5-base-text-simplification
Generating outputs...
BLEU: 59.68 | SARI: 53.34

Loading model: eilamc14/pegasus-large-text-simplification
Generating outputs...


BLEU: 59.95 | SARI: 54.22

Loading model: bogdancazan/t5-small-wikilarge-text-simplification
Generating outputs...
BLEU: 57.68 | SARI: 53.09

Loading model: mrm8488/t5-small-finetuned-text-simplification
Generating outputs...


BLEU: 59.99 | SARI: 57.47

Loading model: haining/scientific_abstract_simplification
Generating outputs...
BLEU: 58.58 | SARI: 54.17

Loading model: elvisbakunzi/dyslexia-friendly-text-simplifier
Generating outputs...
BLEU: 55.86 | SARI: 54.10


In [ ]:
print("\n================ MODEL COMPARISON ================\n")

for k, v in results.items():
    print(f"{k}")
    print(f"  Model: {v['model_name']}")
    print(f"  BLEU : {v['bleu']:.2f}")
    print(f"  SARI : {v['sari']:.2f}")
    print()



================ MODEL COMPARISON ================

flan_t5_base
  Model: google/flan-t5-base
  BLEU : 44.71
  SARI : 44.05

bart-base-text-simplification
  Model: eilamc14/bart-base-text-simplification
  BLEU : 52.79
  SARI : 46.78

flan-t5-base-text-simplification
  Model: eilamc14/flan-t5-base-text-simplification
  BLEU : 59.22
  SARI : 52.19

t5-base-text-simplification
  Model: eilamc14/t5-base-text-simplification
  BLEU : 59.68
  SARI : 53.34

pegasus-large-text-simplification
  Model: eilamc14/pegasus-large-text-simplification
  BLEU : 59.95
  SARI : 54.22

t5-small-wikilarge-text-simplification
  Model: bogdancazan/t5-small-wikilarge-text-simplification
  BLEU : 57.68
  SARI : 53.09

mrm8488/t5-small-finetuned-text-simplification
  Model: mrm8488/t5-small-finetuned-text-simplification
  BLEU : 59.99
  SARI : 57.47

haining/scientific_abstract_simplification
  Model: haining/scientific_abstract_simplification
  BLEU : 58.58
  SARI : 54.17

elvisbakunzi/dyslexia-friendly-text-si